In [7]:
import json
from statistics import mean

import numpy as np
import torch
from torch import Tensor
from tqdm.notebook import tqdm

from main import experiment
from modules import NeuralODE, LinearODEF
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from load_equations_with_solutions import get_eqs, parse_equation

In [9]:
eqs = get_eqs()

In [10]:
def test_du(true_ode_tensor_values):
    ode_true = NeuralODE(LinearODEF(Tensor(true_ode_tensor_values)))
    ode_trained = NeuralODE(LinearODEF(torch.randn(2, 2) / 2.))

    losses, trained = experiment(ode_true, ode_trained, 500, "test", xlim=(-100, 100), ylim=(-100, 100),
                                 plot_freq=100000)

    return losses, trained.func.lin.weight.detach().numpy().tolist()

In [38]:
exp_result = []

In [45]:
for eq in tqdm(eqs):
    if len(eq['points']) == 1:
        eq_json = {
            'equation': eq,
            'mean_loss_in_all': None,
            'full_losses': None,
            'coeffs_diff': None,
        }
        coefs = parse_equation(eq['equation'])
        losses = []
        _full_losses = []
        coeffs_diff = []
        
        for _ in range(5):
            coefs = np.array(coefs) / 10
            # mx = coefs.max()
            # if mx == 0:
            #     continue
            # coefs = coefs / mx
            ls, t = test_du(coefs.tolist())
            losses.append(ls[-1])
            _full_losses.append(ls)
            coeffs_diff.append(float(np.abs(np.array(t) - coefs).mean()))
            
            
        eq_json['mean_loss_in_all'] = mean(losses)
        eq_json['full_losses'] = _full_losses
        # print(coeffs_diff)
        eq_json['coeffs_diff'] = mean(coeffs_diff)
        exp_result.append(eq_json)
        
        

  0%|          | 0/60 [00:00<?, ?it/s]

In [22]:
exp_result

[{'equation': {'equation': '"3 * x - y", "13 * x - 3 * y"',
   'points': [{'point': '\\( \\left( 0, 0 \\right) \\)',
     'point_type': 'Центр',
     'stability': 'Устойчиво по Ляпунову'}]},
  'mean_loss_in_all': 0.2142061256337911,
  'full_losses': [[0.6484964489936829,
    0.11031060665845871,
    1.5109071731567383,
    0.4107970595359802,
    0.8847870230674744,
    0.5349416732788086,
    1.9502216577529907,
    0.3261926472187042,
    1.1137751340866089,
    1.2075555324554443,
    1.5426373481750488,
    0.23105600476264954,
    0.5543418526649475,
    1.0386064052581787,
    2.0836305618286133,
    1.0671944618225098,
    2.0466225147247314,
    0.9733133912086487,
    1.2529323101043701,
    0.12080888450145721,
    0.9948158264160156,
    1.0419743061065674,
    0.9503519535064697,
    1.000390887260437,
    0.030753303319215775,
    1.3460017442703247,
    0.989302933216095,
    0.3890615701675415,
    0.2972450852394104,
    0.025176705792546272,
    1.4247961044311523,
   

In [46]:
d = {}
w = {}
for exp in exp_result:
    point_type = exp['equation']['points'][0]['point_type']
    d.setdefault(point_type, []).append(exp['mean_loss_in_all'])
    w.setdefault(point_type, []).append(exp['coeffs_diff'])
    

In [47]:
w

{'Центр': [0.20418653156733513,
  0.6050478093995154,
  0.23580020727140083,
  0.3452129554220792,
  0.33450814406448603,
  4.155639959964901,
  4.449532417522278,
  2.062854243442416,
  4.187049241922796,
  3.180434597376734],
 'Неустойчивый узел': [0.2752994577114135,
  0.27441147556751966,
  0.1951092534700334,
  0.2708180595436692,
  0.3087477087676525,
  nan,
  nan,
  nan,
  nan,
  nan],
 'Неустойчивый вырожденный узел': [0.2526134374089688,
  0.3553250503612645,
  0.39581378765949604,
  0.2246790062625557,
  0.3077340423476696,
  nan,
  nan,
  nan,
  2.107459208369255,
  nan],
 'Устойчивый узел': [0.3728000461998582,
  0.2895503702019751,
  0.37286720774783194,
  0.25121319423341754,
  0.3059667200685144,
  5.106448029913008,
  3.3832540018483996,
  4.423426864808425,
  2.9657820906490087,
  4.26599677815102],
 'Седло': [0.36365424141946434,
  0.3001808147084713,
  0.40485771183001995,
  0.21817016998824476,
  0.42798260669323057,
  2.491095342300832,
  nan,
  nan,
  nan,
  2.984

In [48]:
d

{'Центр': [0.06135363720241003,
  0.40291953763808125,
  0.0001665062489337288,
  0.1439433205407113,
  0.06227285016066162,
  38.45338749885559,
  126.11409492492676,
  1.4243281587958336,
  9596952750.85189,
  92840.64674448967],
 'Неустойчивый узел': [318218226892.8042,
  19.41665525501012,
  2815205.0001479103,
  611913092300.8003,
  71.83249621875439,
  nan,
  nan,
  nan,
  nan,
  nan],
 'Неустойчивый вырожденный узел': [815.4901070352993,
  3.5937299586847073,
  8.203348139976152,
  0.006781069049611688,
  285.7333995084744,
  nan,
  nan,
  nan,
  2.2714822923391933e+24,
  nan],
 'Устойчивый узел': [0.01487696411786601,
  0.0005912519001867623,
  0.0033257661445531992,
  0.00045187788200564685,
  0.0007210602241684682,
  0.00011492910125525669,
  0.00018865884048864246,
  0.0017265619622776284,
  0.00015413269866257907,
  8.981086866697297e-05],
 'Седло': [0.0005266423569992185,
  2.921094903475023,
  0.046103120266343466,
  0.0014704054832691328,
  0.01066219639033079,
  1.01041

In [34]:
print(json.dumps(d, indent=4, ensure_ascii=False))

{
    "Центр": [
        0.02589100170880556,
        0.09998283628374338,
        0.2127974928007461,
        0.2667751465924084,
        0.0011384612793335692
    ],
    "Неустойчивый узел": [
        715550332948.0416,
        6.757687407198197e+23,
        3.166043676840394e+24,
        76431514031.1923,
        2.4369201288038276e+21
    ],
    "Неустойчивый вырожденный узел": [
        129854305.00833511,
        209483.35497817994,
        8435757.795367012,
        70977942.57872958,
        1.1817111200739812e+18
    ],
    "Устойчивый узел": [
        0.0017452298401622103,
        0.0008046652015764266,
        0.002662430485361256,
        0.0006713342212606221,
        0.0001287930383114144
    ],
    "Седло": [
        9.01165234214859,
        1019754412956.8485,
        1416683664839.2583,
        1953944929212.303,
        1.5715001756878337
    ],
    "Неустойчивый фокус": [
        3.659520207830349e+18,
        5.176618484340259e+25,
        1388706453.686211,
     

In [35]:
print(json.dumps(w, indent=4, ensure_ascii=False))

{
    "Центр": [
        0.13954938693115346,
        0.3557106517255306,
        0.40702366031706333,
        0.39655880366380397,
        0.06333425790071487
    ],
    "Неустойчивый узел": [
        0.6456619139139851,
        0.6093563063070178,
        0.6923121535032988,
        0.43717246130108833,
        0.4790329985693097
    ],
    "Неустойчивый вырожденный узел": [
        0.7311246503693317,
        0.5113031911291183,
        0.5452625658673544,
        0.7612487665067115,
        0.4437970201640079
    ],
    "Устойчивый узел": [
        0.7283369203630302,
        1.229593244396771,
        0.6381918476894497,
        1.2199307354167104,
        0.9970851300749928
    ],
    "Седло": [
        0.7335521968081593,
        0.790334769586722,
        0.8096668332815171,
        0.4630445994436741,
        0.5871810141950846
    ],
    "Неустойчивый фокус": [
        1.0777441657148301,
        2.9564281111583113,
        0.5498707785271109,
        0.6441514573991298,
    

In [49]:
json.dump(d, open('exp_result2.json', 'w'), indent=4, ensure_ascii=False)

In [50]:
json.dump(w, open('exp_result2w.json', 'w'), indent=4, ensure_ascii=False)